In [8]:
# ... existing code ...

# Install pydub and ffmpeg
!pip install pydub
!pip install ffmpeg-python

# Define mp4 and mp3 file paths
mp4_file = '../video/Scoring -20250219_153216-Meeting Recording.mp4'
mp3_file = mp4_file.replace('.mp4', '.mp3')  # Define mp3_file variable

# Convert mp4 to mp3 using ffmpeg directly
import subprocess

try:
    # Use ffmpeg to extract audio from video
    subprocess.call(['ffmpeg', '-i', mp4_file, '-q:a', '0', '-map', 'a', mp3_file, '-y'])
    print(f"Successfully converted {mp4_file} to {mp3_file}")
except Exception as e:
    print(f"Error converting file: {e}")

# ... existing code ...

# Now use OpenAI's Whisper for transcription
from openai import OpenAI
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

# Use the mp3_file we created earlier
try:
    audio_file = open(mp3_file, "rb")
    transcription = client.audio.transcriptions.create(
        model="whisper-1", 
        file=audio_file
    )
    print(transcription.text)
    # Don't forget to close the file
    audio_file.close()
except Exception as e:
    print(f"Error during transcription: {e}")

  Obtaining dependency information for ffmpeg-python from https://files.pythonhosted.org/packages/d7/0c/56be52741f75bad4dc6555991fabd2e07b432d333da82c11ad701123888a/ffmpeg_python-0.2.0-py3-none-any.whl.metadata
Successfully converted ../video/Scoring -20250219_153216-Meeting Recording.mp4 to ../video/Scoring -20250219_153216-Meeting Recording.mp3
Ня спуснах сега, вие трябва да добрите ме. М? Да. Не се мютнахте нещо. То самата мютва. А, да. То те мютва, да, да. Кирил, Тони, мютнави. Мютнави. Не ме виждаме. Не се управи при мен. На микрофонците си цъкнах и те ме попита тогава. Защото не ми излигаше по-папа. Са гласа ми си. Ето и турията е готова. Ами не, на мен ми се прави в някаква страница, която постоя дада ерор. Да, и на мен първо, ама я затворих веднага и после на микрофонците си цъкнах. Ами ти, Митко, али беш записвал? Ами правихме такива срещи там по... В началото казах. Но не помня дали се пробва да ги изтеглям. Да ги даунлодвам. А тогава ще трябва да го даунлоднем и за да го про

In [ ]:
# Load model directly
from transformers import AutoProcessor, AutoModelForCTC

processor = AutoProcessor.from_pretrained("DrishtiSharma/wav2vec2-large-xls-r-300m-bg-d2")
model = AutoModelForCTC.from_pretrained("DrishtiSharma/wav2vec2-large-xls-r-300m-bg-d2")